In [1]:
%matplotlib notebook

import pickle
import sys
import numpy as np
import import_ipynb
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.sparse.linalg import eigsh
from IPython.display import SVG, display
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.animation import FuncAnimation

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from fem_processing import vectorial_matrices_assembly as assembly
    from fem_processing import master_domain as master
    from fem_pos_processing import graph_results as graph
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")

Project root added to sys.path: C:\Users\adilt\OneDrive\05_GIT\PPGEE-2024.2-MEF
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!


# Project 4: Rectangular Guide
# 3-noded Linear Triangular Elements, $P_1$

In [2]:
FINITE_ELEMENT = ("Tetrahedron", 1)
BOUNDARY = [{'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}]
MATERIAL = [{'tag': 301, 'name': 'free_space', 'ur': 1, 'er': 1}]

# Create mesh from file geometry
mesh_data = ps.create_rectangular_cavity(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=1, view_mesh=False)

# Applying physical properties to the mesh
# mesh_data = ps.apply_physics(mesh_data)

# # Salvar mesh_data com pickle
# with open('mesh_data.pkl', 'wb') as f:
#     pickle.dump(mesh_data, f)

Malha salva em pre_processing/mesh/rectangular_cavity_domain_Tetrahedron1.msh
Model rectangular_cavity (3D)
Info     : 27 geometric entities
Info     : 2 physical groups
Info     : 14 nodes in total
Info     : 49 edges in total
Info     : 24 3-D elements in total


In [3]:
mesh_data['cell']

{1: {'tag': 45,
  'conn': [11, 12, 10, 13],
  'conn_sorted': [10, 11, 12, 13],
  'conn_edge': [43, 10, 6, 8, 7, 27],
  'geo': {'centroid': None, 'dim': None},
  'contour': {'type': None, 'conn_contour': None},
  'material': {'tag': 301, 'name': 'free_space', 'ur': 1, 'er': 1}},
 2: {'tag': 46,
  'conn': [11, 12, 14, 10],
  'conn_sorted': [10, 11, 12, 14],
  'conn_edge': [43, 10, 12, 8, 16, 5],
  'geo': {'centroid': None, 'dim': None},
  'contour': {'type': None, 'conn_contour': None},
  'material': {'tag': 301, 'name': 'free_space', 'ur': 1, 'er': 1}},
 3: {'tag': 47,
  'conn': [14, 12, 11, 9],
  'conn_sorted': [9, 11, 12, 14],
  'conn_edge': [3, 2, 4, 8, 16, 5],
  'geo': {'centroid': None, 'dim': None},
  'contour': {'type': None, 'conn_contour': None},
  'material': {'tag': 301, 'name': 'free_space', 'ur': 1, 'er': 1}},
 4: {'tag': 48,
  'conn': [9, 12, 11, 13],
  'conn_sorted': [9, 11, 12, 13],
  'conn_edge': [3, 2, 1, 8, 7, 27],
  'geo': {'centroid': None, 'dim': None},
  'contour'

In [4]:
mesh_data['nodes']

{1: {'xg': (0.0, 0.0, 1.0),
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 2: {'xg': (0.0, 0.0, 0.0),
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 3: {'xg': (0.0, 1.0, 1.0),
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 4: {'xg': (0.0, 1.0, 0.0),
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 5: {'xg': (1.0, 0.0, 1.0),
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 6: {'xg': (1.0, 0.0, 0.0),
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 7: {'xg': (1.0, 1.0, 1.0),
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 8: {'xg': (1.0, 1.0, 0.0),
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 9: {'xg': (0.0, 0.5, 0.5),
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 10: {'xg': (1.0, 0.5, 0.5),
  'bc': {'tag': 201, 'type': 'Dirichlet', 'v

In [5]:
mesh_data['boundary']

{1: {'tag': 21,
  'conn': [2, 1, 9],
  'conn_sorted': [1, 2, 9],
  'conn_edge': [38, 34, 26],
  'geo': {'centroid': None, 'dim': None},
  'contour': {'type': None, 'conn_contour': None},
  'boundary': {'tag': 201,
   'type': 'Dirichlet',
   'value': 0.0,
   'name': 'nxE=0'}},
 2: {'tag': 22,
  'conn': [1, 3, 9],
  'conn_sorted': [1, 3, 9],
  'conn_edge': [30, 34, 42],
  'geo': {'centroid': None, 'dim': None},
  'contour': {'type': None, 'conn_contour': None},
  'boundary': {'tag': 201,
   'type': 'Dirichlet',
   'value': 0.0,
   'name': 'nxE=0'}},
 3: {'tag': 23,
  'conn': [4, 2, 9],
  'conn_sorted': [2, 4, 9],
  'conn_edge': [25, 26, 15],
  'geo': {'centroid': None, 'dim': None},
  'contour': {'type': None, 'conn_contour': None},
  'boundary': {'tag': 201,
   'type': 'Dirichlet',
   'value': 0.0,
   'name': 'nxE=0'}},
 4: {'tag': 24,
  'conn': [3, 4, 9],
  'conn_sorted': [3, 4, 9],
  'conn_edge': [37, 42, 15],
  'geo': {'centroid': None, 'dim': None},
  'contour': {'type': None, 'conn

In [6]:
mesh_data['edges']

{1: {'conn': [9, 13],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 2: {'conn': [9, 12],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 3: {'conn': [9, 11],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 4: {'conn': [9, 14],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 5: {'conn': [12, 14],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 6: {'conn': [10, 13],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 7: {'conn': [11, 13],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 8: {'conn': [11, 12],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 9: {'conn': [1, 14],
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 10: {'conn': [10, 12],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 11: {'conn': [

In [7]:
dirichlet_edges = {key: edge for key, edge in mesh_data['edges'].items() if edge['bc']['tag'] == 201}
dirichlet_edges

{9: {'conn': [1, 14],
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 11: {'conn': [8, 13],
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 13: {'conn': [6, 13],
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 14: {'conn': [5, 14],
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 15: {'conn': [4, 9],
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 17: {'conn': [5, 10],
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 18: {'conn': [2, 13],
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 19: {'conn': [3, 7],
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 20: {'conn': [4, 8],
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 21: {'conn': [8, 10],
  'bc': {'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 22: {'conn': [3, 14],
 

In [8]:
free_edges = {key: edge for key, edge in mesh_data['edges'].items() if edge['bc']['tag'] == None}
free_edges

{1: {'conn': [9, 13],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 2: {'conn': [9, 12],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 3: {'conn': [9, 11],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 4: {'conn': [9, 14],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 5: {'conn': [12, 14],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 6: {'conn': [10, 13],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 7: {'conn': [11, 13],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 8: {'conn': [11, 12],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 10: {'conn': [10, 12],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 12: {'conn': [10, 14],
  'bc': {'tag': None, 'type': 'Free', 'value': None, 'name': 'free_edge'}},
 16: {'conn'

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.